In [1]:
import numpy as np
import warnings 
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/andro.arff')
data = pd.DataFrame(data)
data.head()

,Window0-Att0,Window0-Att1,Window0-Att2,Window0-Att3,Window0-Att4,Window0-Att5,Window1-Att0,Window1-Att1,Window1-Att2,Window1-Att3,...,Window4-Att2,Window4-Att3,Window4-Att4,Window4-Att5,Target,Target_2,Target_3,Target_4,Target_5,Target_6
0,11.84,5.08,49.0,32.2,16.5,1.40,12.01,5.09,48.0,32.0,...,45.0,29.7,38.5,3.27,15.17,5.14,46.0,30.4,46.8,3.74
1,12.01,5.09,48.0,32.0,14.8,1.25,12.89,5.10,48.0,31.9,...,46.0,30.1,42.6,3.55,14.85,5.11,47.0,31.2,46.8,3.78
2,12.89,5.10,48.0,31.9,8.6,0.71,13.95,5.11,47.0,30.8,...,48.0,31.3,46.0,3.79,14.72,5.10,48.0,31.7,42.7,3.40
3,13.95,5.11,47.0,30.8,20.2,1.64,13.75,5.13,45.0,29.7,...,47.0,31.1,46.7,3.79,15.28,5.11,48.0,32.0,42.5,3.38
4,13.75,5.13,45.0,29.7,38.5,3.27,13.34,5.13,46.0,30.1,...,45.0,29.7,45.1,3.75,16.15,5.13,47.0,30.8,42.1,3.27


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(49, 30) (49, 6)


In [4]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(39, 30) (10, 30) (39, 6) (10, 6)


In [10]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print('MAE - ',mean_absolute_error(y_train,y_test))
    print('RMSE - ',mean_squared_error(y_train,y_test)**0.5)
    print('R-squared - ',r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 49
    p = 30

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### KNN Regression

In [11]:
from sklearn.neighbors import KNeighborsRegressor

In [12]:
for i in range(1,20):
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train,y_train)
    print(i,":",model.score(X_test,y_test))

1 : 0.7648648146308288
2 : 0.8743852546968308
3 : -1.5968221421786424
4 : -0.8090816589095394
5 : -2.9439569990336936
6 : -4.915455202259749
7 : -3.665895613222165
8 : -5.193309590776445
9 : -6.406461754268956
10 : -7.509969859295079
11 : -6.243704783268568
12 : -7.258495133943384
13 : -8.146922330226062
14 : -7.027507875839472
15 : -6.117151447048479
16 : -6.817950070349255
17 : -6.12280691236671
18 : -5.552110095433921
19 : -5.0554827390837485


In [13]:
model = KNeighborsRegressor(n_neighbors=9)
model.fit(X_train,y_train)
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  4.445546296296296
RMSE -  8.993273809449391
R-squared -  -6.406461754268956
Adjusted R-squared -  -18.750564678050548


### Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
model = RandomForestRegressor(n_estimators=5)
model.fit(X_train,y_train)
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  2.6038833333333335
RMSE -  5.771307201723598
R-squared -  -0.3075251884327593
Adjusted R-squared -  -2.4867338358206914


In [17]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
model = Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(64,activation='relu'))
model.add(Dense(6,activation='linear'))
model.compile(optimizer="adam",loss='mse')

In [19]:
model.fit(X_train,y_train,epochs=500,batch_size=50)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
39/39 [==============================] - 0s - loss: 1.2567
Epoch 2/500
39/39 [==============================] - 0s - loss: 1.1056
Epoch 3/500
39/39 [==============================] - 0s - loss: 0.9761
Epoch 4/500
39/39 [==============================] - 0s - loss: 0.8684
Epoch 5/500
39/39 [==============================] - 0s - loss: 0.7811
Epoch 6/500
39/39 [==============================] - 0s - loss: 0.7127
Epoch 7/500
39/39 [==============================] - 0s - loss: 0.6602
Epoch 8/500
39/39 [==============================] - 0s - loss: 0.6198
Epoch 9/500
39/39 [==============================] - 0s - loss: 0.5884
Epoch 10/500
39/39 [==============================] - 0s - loss: 0.5619
Epoch 11/500
39/39 [==============================] - 0s - loss: 0.5382
Epoch 12/500
39/39 [==============================] - 0s - loss: 0.5154
Epoch 13/500
39/39 [=====

39/39 [==============================] - 0s - loss: 0.0276
Epoch 105/500
39/39 [==============================] - 0s - loss: 0.0270
Epoch 106/500
39/39 [==============================] - 0s - loss: 0.0264
Epoch 107/500
39/39 [==============================] - 0s - loss: 0.0259
Epoch 108/500
39/39 [==============================] - 0s - loss: 0.0253
Epoch 109/500
39/39 [==============================] - 0s - loss: 0.0248
Epoch 110/500
39/39 [==============================] - 0s - loss: 0.0243
Epoch 111/500
39/39 [==============================] - 0s - loss: 0.0238
Epoch 112/500
39/39 [==============================] - 0s - loss: 0.0234
Epoch 113/500
39/39 [==============================] - 0s - loss: 0.0229
Epoch 114/500
39/39 [==============================] - 0s - loss: 0.0225
Epoch 115/500
39/39 [==============================] - 0s - loss: 0.0220
Epoch 116/500
39/39 [==============================] - 0s - loss: 0.0216
Epoch 117/500
39/39 [==============================] - 0s - loss:

39/39 [==============================] - 0s - loss: 0.0043
Epoch 217/500
39/39 [==============================] - 0s - loss: 0.0043
Epoch 218/500
39/39 [==============================] - 0s - loss: 0.0042
Epoch 219/500
39/39 [==============================] - 0s - loss: 0.0041
Epoch 220/500
39/39 [==============================] - 0s - loss: 0.0041
Epoch 221/500
39/39 [==============================] - 0s - loss: 0.0040
Epoch 222/500
39/39 [==============================] - 0s - loss: 0.0039
Epoch 223/500
39/39 [==============================] - 0s - loss: 0.0039
Epoch 224/500
39/39 [==============================] - 0s - loss: 0.0038
Epoch 225/500
39/39 [==============================] - 0s - loss: 0.0038
Epoch 226/500
39/39 [==============================] - 0s - loss: 0.0037
Epoch 227/500
39/39 [==============================] - 0s - loss: 0.0036
Epoch 228/500
39/39 [==============================] - 0s - loss: 0.0036
Epoch 229/500
39/39 [==============================] - 0s - loss:

39/39 [==============================] - 0s - loss: 6.5128e-04
Epoch 328/500
39/39 [==============================] - 0s - loss: 6.4009e-04
Epoch 329/500
39/39 [==============================] - 0s - loss: 6.2724e-04
Epoch 330/500
39/39 [==============================] - 0s - loss: 6.1602e-04
Epoch 331/500
39/39 [==============================] - 0s - loss: 6.0744e-04
Epoch 332/500
39/39 [==============================] - 0s - loss: 5.9845e-04
Epoch 333/500
39/39 [==============================] - 0s - loss: 5.8812e-04
Epoch 334/500
39/39 [==============================] - 0s - loss: 5.7796e-04
Epoch 335/500
39/39 [==============================] - 0s - loss: 5.6817e-04
Epoch 336/500
39/39 [==============================] - 0s - loss: 5.5690e-04
Epoch 337/500
39/39 [==============================] - 0s - loss: 5.4805e-04
Epoch 338/500
39/39 [==============================] - 0s - loss: 5.4009e-04
Epoch 339/500
39/39 [==============================] - 0s - loss: 5.3189e-04
Epoch 340/500

39/39 [==============================] - 0s - loss: 1.2359e-04
Epoch 434/500
39/39 [==============================] - 0s - loss: 1.2186e-04
Epoch 435/500
39/39 [==============================] - 0s - loss: 1.1990e-04
Epoch 436/500
39/39 [==============================] - 0s - loss: 1.1826e-04
Epoch 437/500
39/39 [==============================] - 0s - loss: 1.1666e-04
Epoch 438/500
39/39 [==============================] - 0s - loss: 1.1483e-04
Epoch 439/500
39/39 [==============================] - 0s - loss: 1.1306e-04
Epoch 440/500
39/39 [==============================] - 0s - loss: 1.1140e-04
Epoch 441/500
39/39 [==============================] - 0s - loss: 1.0968e-04
Epoch 442/500
39/39 [==============================] - 0s - loss: 1.0809e-04
Epoch 443/500
39/39 [==============================] - 0s - loss: 1.0652e-04
Epoch 444/500
39/39 [==============================] - 0s - loss: 1.0500e-04
Epoch 445/500
39/39 [==============================] - 0s - loss: 1.0348e-04
Epoch 446/500

In [20]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  1.8298518228530891
RMSE -  4.114252343682089
R-squared -  0.1378501018151234
Adjusted R-squared -  -1.299066395159671
